In [2]:
#imports
import pandas as pd
import sqlite3


In [3]:
#load data
orders = pd.read_csv("data/orders.csv")
users = pd.read_json("data/users.json")

conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

with open("data/restaurants.sql", "r") as f:
    sql_script = f.read()

cursor.executescript(sql_script)
conn.commit()

restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)


In [5]:
#merge
df = pd.merge(orders, users, on="user_id", how="left")
final_df = pd.merge(df, restaurants, on="restaurant_id", how="left")

final_df.to_csv("output/final_food_delivery_dataset.csv", index=False)
final_df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [6]:
#analysis
final_df["order_date"] = pd.to_datetime(final_df["order_date"], dayfirst=True)

gold_df = final_df[final_df["membership"] == "Gold"]

q1 = gold_df.groupby("city")["total_amount"].sum().sort_values(ascending=False).head(1)
q2 = final_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False).head(1)
q3 = final_df.groupby("user_id")["total_amount"].sum().gt(1000).sum()

final_df["rating_range"] = pd.cut(final_df["rating"], bins=[0,3,4,5], labels=["Low","Medium","High"])
q4 = final_df.groupby("rating_range")["total_amount"].sum().sort_values(ascending=False).head(1)

q5 = gold_df.groupby("city")["total_amount"].mean().sort_values(ascending=False).head(1)

cuisine_stats = final_df.groupby("cuisine").agg(
    total_revenue=("total_amount","sum"),
    restaurant_count=("restaurant_id","nunique")
)
q6 = cuisine_stats.sort_values(["restaurant_count","total_revenue"], ascending=[True, False]).head(1)

q7 = round((len(gold_df)/len(final_df))*100)

rest_stats = final_df.groupby("restaurant_name_y").agg(
    avg_order=("total_amount","mean"),
    total_orders=("order_id","count")
)
q8 = rest_stats[rest_stats["total_orders"]<20].sort_values("avg_order", ascending=False).head(1)

q9 = final_df.groupby(["city","cuisine"])["total_amount"].sum().sort_values(ascending=False).head(1)

final_df["quarter"] = final_df["order_date"].dt.to_period("Q")
q10 = final_df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False).head(1)

q1, q2, q3, q4, q5, q6, q7, q8, q9, q10


(city
 Chennai    1080909.79
 Name: total_amount, dtype: float64,
 cuisine
 Mexican    808.021344
 Name: total_amount, dtype: float64,
 np.int64(2544),
 rating_range
 High    4157357.01
 Name: total_amount, dtype: float64,
 city
 Chennai    808.45908
 Name: total_amount, dtype: float64,
          total_revenue  restaurant_count
 cuisine                                 
 Chinese     1930504.65               120,
 50,
                      avg_order  total_orders
 restaurant_name_y                           
 Restaurant_294     1040.222308            13,
 city       cuisine
 Bangalore  Mexican    571004.61
 Name: total_amount, dtype: float64,
 quarter
 2023Q3    2037385.1
 Freq: Q-DEC, Name: total_amount, dtype: float64)

In [7]:
# Total orders by Gold members
gold_orders = final_df[final_df["membership"] == "Gold"]
gold_orders_count = len(gold_orders)

# Total revenue from Hyderabad (rounded)
hyd_revenue = round(final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum())

# Distinct users who placed at least one order
distinct_users = final_df["user_id"].nunique()

# Average order value for Gold members (rounded to 2 decimals)
avg_gold_order = round(final_df[final_df["membership"] == "Gold"]["total_amount"].mean(), 2)

# Orders for restaurants with rating ≥ 4.5
high_rating_orders = len(final_df[final_df["rating"] >= 4.5])

# Orders in the top revenue city among Gold members
gold_city_revenue = final_df[final_df["membership"] == "Gold"] \
                        .groupby("city")["total_amount"].sum() \
                        .sort_values(ascending=False)

top_gold_city = gold_city_revenue.index[0]

orders_top_gold_city = len(
    final_df[(final_df["membership"] == "Gold") & (final_df["city"] == top_gold_city)]
)

gold_orders_count, hyd_revenue, distinct_users, avg_gold_order, high_rating_orders, top_gold_city, orders_top_gold_city


(4987, 1889367, 2883, np.float64(797.15), 3374, 'Chennai', 1337)

In [8]:
#FIB
join_orders_users = "user_id"
restaurant_data_format = "SQL (.sql file)"
final_rows = len(final_df)  # Same as orders.csv
missing_user_values = "NaN / Null"
merge_function = "pd.merge()"
membership_source = "users.json"
join_orders_restaurants = "restaurant_id"
food_type_column = "cuisine"
user_multiple_orders = "Multiple times (one row per order)"

join_orders_users, restaurant_data_format, final_rows, missing_user_values, merge_function, membership_source, join_orders_restaurants, food_type_column, user_multiple_orders


('user_id',
 'SQL (.sql file)',
 10000,
 'NaN / Null',
 'pd.merge()',
 'users.json',
 'restaurant_id',
 'cuisine',
 'Multiple times (one row per order)')